In [ ]:
from collections import defaultdict
import numpy as np 
import os
import re

class Tranf_encode:
    def __init__(self,max_doc_length,min_df):
        self._vocab=[]
        self._max_doc_length=max_doc_length
        self._min_df=min_df
    
    def isappear(self,word,wordslist):
        l=0
        r=len(wordslist)-1
        while(l<=r):
            m=int((r+l)/2)
            if(word==wordslist[m]):
                return m
            else:
                if(word<wordslist[m]):
                    r=m-1
                else:
                    l=m+1
        return -1

    def fit(self,path_train):
        def get_data_and_vocab(path_train):
            data=[]
            vocab_count=defaultdict(int)
            list_folder=os.listdir(path_train)
            list_folder.sort()
            i_folder=0
            # clear data and find vocabulary
            for folder in list_folder:
                path_folder=path_train+'/'+folder
                list_doc=os.listdir(path_folder)
                list_doc.sort()
                for doc in list_doc:
                    path_file=path_folder+'/'+doc
                    with open(path_file,'r',errors='ignore') as f:
                        text=f.read()
                    words=re.split('\W+',text)
                    for word in words:
                        vocab_count[word]+=1
                    content=' '.join(words)
                    ifo_doc=str(i_folder)+'<fff>'+str(doc)+'<fff>'+content
                    data.append(ifo_doc)
                i_folder+=1
            
            self._vocab=[word for word in vocab_count.keys()
                                if vocab_count[word]>self._min_df]
            self._vocab.sort()
            return data
        
        data_train=get_data_and_vocab(path_train)
        data_train_encode=self.get_encode(data_train)
        return data_train_encode
        
    def get_encode(self,data):
        vocab_id=dict([word, self._vocab.index(word)+2] for word in self._vocab) 
        padding_ID=0
        unknown_ID=1
        data_encode=[]
        for doc in data:
            label,doc_id,text=doc.split('<fff>')
            words=text.split()[:self._max_doc_length]
            doc_encode=[]
            for word in words:
                if self.isappear(word,self._vocab)!=-1:
                    doc_encode.append(str(vocab_id[word]))
                else:
                    doc_encode.append(str(unknown_ID))
            if(len(words)<self._max_doc_length):
                for _ in range(self._max_doc_length-len(words)):
                    doc_encode.append(str(padding_ID))
            data_encode.append(label+'<fff>'+doc_id+'<fff>'+str(len(words))+'<fff>'+' '.join(doc_encode))
        
        return data_encode
        
    def fit_tranf(self,path_in):
        data=[]
        list_folder=os.listdir(path_in)
        list_folder.sort()
        i_folder=0
        for folder in list_folder:
            path_folder=path_in+'/'+folder
            list_doc=os.listdir(path_folder)
            list_doc.sort()
            for doc in list_doc:
                path_doc=path_folder+'/'+doc
                with open(path_doc,'r',errors='ignore') as f:
                    text=f.read()
                words=re.split('\W+',text)
                content=' '.join(words)
                ifo_doc=str(i_folder)+'<fff>'+doc+'<fff>'+content
                data.append(ifo_doc)
            i_folder+=1
        
        data_encode=self.get_encode(data)
        return data_encode
    
    def get_vocab(self):
        return self._vocab

model=Tranf_encode(500,10)
train=model.fit('/content/drive/My Drive/Datasets/20news-bydate-train')
test=model.fit_tranf('/content/drive/My Drive/Datasets/20news-bydate-test')
vocab=model.get_vocab()

with open('/content/drive/My Drive/Datasets/trainencode1.txt','w') as f:
    f.write('\n'.join(train))
with open('/content/drive/My Drive/Datasets/testencode1.txt','w') as f:
    f.write('\n'.join(test))
with open('/content/drive/My Drive/Datasets/vocab1.txt','w') as f:
    f.write('\n'.join(vocab))
